In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-08 03:08:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.76MB/s    in 0.2s    

2022-03-08 03:08:06 (4.76 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("az_watches").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://emily-data-bootcamp.s3.amazonaws.com/amazon_reviews_us_Watches_v1_00.tsv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv"), inferSchema=True, sep = '\t')
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [5]:
# Count the number of records (rows) in the dataset.

df.count()

960872

In [6]:
# check the data types 
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
#Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name.

# drop incomplete or duplicate rows if there are any

df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())


960679
960679


In [8]:
# select columns to create the dfs that match those in our sql schemas 

# for the review_id_table table
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R35B5RCH7MRCST|   25027442|B00VEAZF0U|     197150604| 2015-08-30|
| RTQVRZNZJDL9Y|   43727482|B0027EGP2Y|     425229026| 2015-08-30|
|R3ESEVNALOWRIK|   10320069|B00I0M684C|     691316619| 2015-08-30|
| RSRAS3IQEPBHT|   21666519|B00791QYMQ|     494485833| 2015-08-30|
|R1S475C1BPV5AP|   16628764|B00H9JP3O8|     783174867| 2015-08-30|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [9]:
# change data type of the review_date column (other columns are of the proper data type)
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType

review_id_table = review_id_table.withColumn("review_date",col("review_date").cast(DateType()))
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
# for the products table
products = df.select(["product_id", "product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00VEAZF0U|Triple Watch Wind...|
|B0027EGP2Y|Colibri Pocket Wa...|
|B00I0M684C|Womens Watch Set ...|
|B00791QYMQ|Casio Women's LRW...|
|B00H9JP3O8|Pangaea Quad Auto...|
+----------+--------------------+
only showing top 5 rows



In [11]:
# dropping dupes for products table specifically
products = products.dropDuplicates(['product_id', 'product_title'])

In [12]:
# double checking that there are indeed no dupes 
products \
    .groupby(['product_id']) \
    .count() \
    .where('count > 1') \
    .show()

+----------+-----+
|product_id|count|
+----------+-----+
+----------+-----+



In [13]:
# and for the customers table 

customers = df.select(["customer_id"])

customers = customers.groupby("customer_id").agg({"customer_id": "count"})
customers.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|    4089580|                 1|
|   22754948|                 1|
|   20097895|                 1|
|    2422277|                 4|
|   14404180|                 5|
+-----------+------------------+
only showing top 5 rows



In [14]:
# rename the count customer id field to customer_count to match the sql schema

customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show(5)


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    4089580|             1|
|   22754948|             1|
|   20097895|             1|
|    2422277|             4|
|   14404180|             5|
+-----------+--------------+
only showing top 5 rows



In [15]:
# double checking to make sure that customer_count is of the proper data type
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [16]:
# dropping dupes for customers table specifically
customers = customers.dropDuplicates(['customer_id'])

# double checking that there are indeed no dupes 
customers \
    .groupby(['customer_id']) \
    .count() \
    .where('count > 1') \
    .show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



In [19]:
# Load the DataFrames that correspond to tables into an RDS instance.

mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}


In [20]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [22]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)